# Assistants

Sabemos que los LLMs son grandes cajas negras que interpretan el lenguaje natural y nos pueden dar respuestas tambien en nuestro lenguaje de acuerdo a la informacion que tienen comprimida adentro de acuerdo a su entrenamiento.

Que pasa si a estos LLMs ademas les damos herramientas para que puedan obtener informacion actualizada y verificada del mundo real?

## OpenAI Functions

Veamos como podemos darles estas herramientas a los LLMs. En OpenAI les llaman functions:

In [ ]:
!pip install openai --upgrade --quiet

In [ ]:
import openai

api_key="INGRESA AQUI TU TOKEN DE OPENAI"

In [ ]:
#@title [Opcional] cargar el token desde colab para que no se vea al dar clases {display-mode:"form"}
if api_key == "INGRESA AQUI TU TOKEN DE OPENAI":
    from google.colab import userdata
    api_key = userdata.get('OPENAI_TOKEN')
    openai.api_key = api_key

### OpenAI functions:

In [ ]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="celcius"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "26",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)